In [ ]:
import os
import random
import cv2
import numpy as np
import shutil

# Paths
fine_tuning_dir = 'your-dataset-path'
augmented_dir = 'your-augmented-dataset-path'
os.makedirs(augmented_dir, exist_ok=True)


def read_annotations(annotation_path):
    """Reads bounding box annotations from a .txt file."""
    with open(annotation_path, 'r') as file:
        lines = file.readlines()
    boxes = [list(map(int, line.strip().split(','))) if line.strip() != '0,0,0,0' else None for line in lines]
    return boxes


def apply_standard_augmentations(image):
    """Apply standard augmentations such as brightness, contrast, blurring, and color jittering."""

    # Randomly choose between brightness, contrast, or color jittering
    pixel_adjustment = random.choice(['brightness', 'contrast', 'color_jittering'])

    if pixel_adjustment == 'brightness':
        factor = 0.8 + 0.4 * random.random()
        image = cv2.convertScaleAbs(image, alpha=factor, beta=0)

    elif pixel_adjustment == 'contrast':
        factor = 0.8 + 0.4 * random.random()
        image = cv2.convertScaleAbs(image, alpha=factor, beta=0)

    elif pixel_adjustment == 'color_jittering':
        image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hue_factor = random.uniform(0.9, 1.1)
        sat_factor = random.uniform(0.9, 1.1)
        val_factor = random.uniform(0.9, 1.1)
        image_hsv[:, :, 0] = np.clip(image_hsv[:, :, 0] * hue_factor, 0, 255)
        image_hsv[:, :, 1] = np.clip(image_hsv[:, :, 1] * sat_factor, 0, 255)
        image_hsv[:, :, 2] = np.clip(image_hsv[:, :, 2] * val_factor, 0, 255)
        image = cv2.cvtColor(image_hsv, cv2.COLOR_HSV2BGR)

    # Random blur (this can still run independently)
    if random.random() < 0.5:
        ksize = random.choice([3, 5])
        image = cv2.GaussianBlur(image, (ksize, ksize), 0)

    return image


def replace_corresponding_lines(base_boxes, new_boxes):
    """Replace the base label's empty '0,0,0,0' entries with corresponding new_boxes."""
    updated_boxes = base_boxes.copy()
    for i, box in enumerate(new_boxes):
        if box is not None and updated_boxes[i] is None:  # Replace only if base has no object (0,0,0,0)
            updated_boxes[i] = box
    return updated_boxes


def apply_cutmix(image1, boxes1, image2, boxes2):
    """Apply CutMix augmentation between two images."""
    if image2 is None:
        return image1, boxes1

    h, w, _ = image1.shape
    cut_x = random.randint(int(0.3 * w), int(0.7 * w))
    cut_y = random.randint(int(0.3 * h), int(0.7 * h))

    new_image = image1.copy()
    new_image[cut_y:, cut_x:] = image2[cut_y:, cut_x:]

    new_boxes = [box for box in boxes1 if box is not None and box[0] < cut_x and box[1] < cut_y]
    new_boxes.extend([box for box in boxes2 if box is not None and box[0] >= cut_x and box[1] >= cut_y])

    # Replace corresponding lines from the second image's boxes in the base label
    final_boxes = replace_corresponding_lines(boxes1, boxes2)

    return new_image, final_boxes


def apply_mixup(image1, boxes1, image2, boxes2, alpha=0.5):
    """Apply MixUp augmentation."""
    if image2 is None:
        return image1, boxes1

    h, w = image1.shape[:2]
    image2 = cv2.resize(image2, (w, h))
    new_image = cv2.addWeighted(image1, alpha, image2, 1 - alpha, 0)

    # Replace corresponding lines instead of appending
    final_boxes = replace_corresponding_lines(boxes1, boxes2)

    return new_image, final_boxes


# Rest of the advanced augmentations (Mosaic, Cropping, etc.) remain the same

# Save original, standard, and augmented images
for image_file in os.listdir(fine_tuning_dir):
    if not image_file.endswith('.jpg'):
        continue

    image_path = os.path.join(fine_tuning_dir, image_file)
    annotation_path = os.path.join(fine_tuning_dir, image_file.replace('.jpg', '.txt'))
    image = cv2.imread(image_path)
    boxes = read_annotations(annotation_path)

    # Save original image and annotations
    original_image_name = f'original_{image_file}'
    original_annotation_name = f'original_{image_file.replace(".jpg", ".txt")}'
    cv2.imwrite(os.path.join(augmented_dir, original_image_name), image)
    shutil.copy(annotation_path, os.path.join(augmented_dir, original_annotation_name))

    # Apply standard augmentation
    standard_image = apply_standard_augmentations(image.copy())
    standard_image_name = f'standard_{image_file}'
    standard_annotation_name = f'standard_{image_file.replace(".jpg", ".txt")}'
    cv2.imwrite(os.path.join(augmented_dir, standard_image_name), standard_image)
    shutil.copy(annotation_path, os.path.join(augmented_dir, standard_annotation_name))

    # Advanced augmentation: MixUp, CutMix, etc.
    new_image, new_boxes = None, None
    other_image_path = random.choice(os.listdir(fine_tuning_dir))
    other_image = cv2.imread(os.path.join(fine_tuning_dir, other_image_path))
    other_boxes = read_annotations(os.path.join(fine_tuning_dir, other_image_path.replace('.jpg', '.txt')))

    if random.random() < 0.4:  # CutMix
        new_image, new_boxes = apply_cutmix(image.copy(), boxes, other_image, other_boxes)
    elif random.random() < 0.4:  # MixUp
        new_image, new_boxes = apply_mixup(image.copy(), boxes, other_image, other_boxes)

    if new_image is not None and new_boxes is not None:
        # Ensure 24 lines are kept
        final_boxes = replace_corresponding_lines(boxes, new_boxes)

        # Save augmented image and annotations
        advanced_image_name = f'advanced_{image_file}'
        advanced_annotation_name = f'advanced_{image_file.replace(".jpg", ".txt")}'
        cv2.imwrite(os.path.join(augmented_dir, advanced_image_name), new_image)
        with open(os.path.join(augmented_dir, advanced_annotation_name), 'w') as f:
            for box in final_boxes:
                if box is not None:
                    f.write(f'{box[0]},{box[1]},{box[2]},{box[3]}\n')
                else:
                    f.write('0,0,0,0\n')

print("Data augmentation completed!")